# Pose Landmarker

https://developers.google.com/mediapipe/solutions/vision/pose_landmarker

In [2]:
# !wget -O ./models/pose_landmarker.task -q https://storage.googleapis.com/mediapipe-models/pose_landmarker/pose_landmarker_heavy/float16/1/pose_landmarker_heavy.task

In [7]:
model_path = './models/pose_landmarker.task'

In [10]:
import cv2
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision

In [11]:
from mediapipe import solutions
from mediapipe.framework.formats import landmark_pb2
import numpy as np


def draw_landmarks_on_image(rgb_image, detection_result):
  pose_landmarks_list = detection_result.pose_landmarks
  annotated_image = np.copy(rgb_image)

  # Loop through the detected poses to visualize.
  for idx in range(len(pose_landmarks_list)):
    pose_landmarks = pose_landmarks_list[idx]

    # Draw the pose landmarks.
    pose_landmarks_proto = landmark_pb2.NormalizedLandmarkList()
    pose_landmarks_proto.landmark.extend([
      landmark_pb2.NormalizedLandmark(x=landmark.x, y=landmark.y, z=landmark.z) for landmark in pose_landmarks
    ])
    solutions.drawing_utils.draw_landmarks(
      annotated_image,
      pose_landmarks_proto,
      solutions.pose.POSE_CONNECTIONS,
      solutions.drawing_styles.get_default_pose_landmarks_style())
  return annotated_image

## Image

In [12]:
base_options = python.BaseOptions(model_asset_path=model_path)
options = vision.PoseLandmarkerOptions(
    base_options=base_options,
    output_segmentation_masks=True)
image = mp.Image.create_from_file("./../../../../test_img/pose.jpg")

In [13]:
with vision.PoseLandmarker.create_from_options(options) as detector:
    detection_result = detector.detect(image)
    annotated_image = draw_landmarks_on_image(image.numpy_view(), detection_result)
    cv2.imshow('test', cv2.cvtColor(annotated_image, cv2.COLOR_RGB2BGR))
    cv2.waitKey(3000)
    cv2.destroyAllWindows()

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [14]:
segmentation_mask = detection_result.segmentation_masks[0].numpy_view()
visualized_mask = np.repeat(segmentation_mask[:, :, np.newaxis], 3, axis=2) * 255
cv2.imshow('test', visualized_mask)
cv2.waitKey(3000)
cv2.destroyAllWindows()

## Video

In [15]:
cap = cv2.VideoCapture('./../../../../test_video/pose.mp4')
fps = cap.get(cv2.CAP_PROP_FPS)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
output_vid = cv2.VideoWriter('./../segmentation_output.mp4', cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))

In [16]:
BaseOptions = mp.tasks.BaseOptions
PoseLandmarker = mp.tasks.vision.PoseLandmarker
PoseLandmarkerOptions = mp.tasks.vision.PoseLandmarkerOptions
VisionRunningMode = mp.tasks.vision.RunningMode

# Create a pose landmarker instance with the video mode:
options = PoseLandmarkerOptions(
    base_options=BaseOptions(model_asset_path=model_path),
    running_mode=VisionRunningMode.VIDEO)

In [17]:
with PoseLandmarker.create_from_options(options) as landmarker:
    frame_index = 1
    while True:
        ret, frame = cap.read()
        if ret:
            numpy_frame_from_opencv = np.asarray(frame)
            mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=numpy_frame_from_opencv)
            frame_timestamp_ms = int(1000 * frame_index / fps)
            pose_landmarker_result = landmarker.detect_for_video(mp_image, frame_timestamp_ms)
            
            annotated_image = draw_landmarks_on_image(numpy_frame_from_opencv, pose_landmarker_result)
            output_vid.write(annotated_image)
            
            frame_index += 1
        else:
            # VideoWriterを解放
            output_vid.release()
            cap.release()
            break

## Camera

In [20]:
base_options = python.BaseOptions(model_asset_path=model_path)
options = vision.PoseLandmarkerOptions(
    base_options=base_options,
    output_segmentation_masks=True)

In [21]:
with vision.PoseLandmarker.create_from_options(options) as detector:
    cap = cv2.VideoCapture(0)
    while True:
        ret, frame = cap.read()
        if ret:
            numpy_frame_from_opencv = np.asarray(frame)
            mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=numpy_frame_from_opencv)
            detection_result = detector.detect(mp_image)
            
            if 0 < len(detection_result.pose_landmarks):
                annotated_image = draw_landmarks_on_image(numpy_frame_from_opencv, detection_result)
                cv2.imshow('camera', annotated_image)
            else:
                cv2.imshow('camera', frame)

            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
        else:
            break

    cap.release()
    cv2.destroyAllWindows()